# Pipe Flow Heat Transport PDE with Pyomo

Solving the heat transport equation for incompressible fluid flow in a horizontal pipe:

**Solar collector section ($0 < x ≤ L$):**
$$\frac{\partial T}{\partial t} + v(t) \frac{\partial T}{\partial x} = \alpha \frac{\partial^2 T}{\partial x^2} + \frac{q(t)}{\rho c_p} - \frac{4 h(T - T_{amb})}{D \rho c_p}$$

**Insulated pipe extension at outlet ($L < x ≤ L_{extended}$):**
$$\frac{\partial T}{\partial t} + v(t) \frac{\partial T}{\partial x} = \alpha \frac{\partial^2 T}{\partial x^2}$$

Where:
- $T(x,t)$ : temperature at position x and time t [K]
- $v(t)$ : time-varying fluid velocity [m/s]
- $\alpha$ : thermal diffusivity [m²/s]
- $q(t)$ : time-varying volumetric heat input rate [W/m³]
- $c_p$ : specific heat capacity [J/kg·K]
- $h$ : convective heat transfer coefficient [W/m²·K]
- $D$ : pipe inner diameter [m]
- $T_{amb}$ : ambient temperature [K]

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyomo as pyo

In [ ]:
plot_dir = "plots"

## Single Temperature Flow Model Simulation

In [ ]:
from solar_collector.solar_collector_dae_pyo import (
    ZERO_C,
    create_pipe_flow_model,
    add_pde_constraints, 
    solve_model,
    plot_results,
    print_temp_profiles
)

In [ ]:
# Create and solve the model
print("Creating pipe flow heat transport model...")
model = create_pipe_flow_model(t_final=60.0*5)  # Simulate for 5 minutes

print("Adding PDE constraints...")
model = add_pde_constraints(model)

print("Solving the discretized PDE...")
results = solve_model(model, n_x=110, n_t=50, tol=1e-6)

print(f"Solver status: {results.solver.status}")
print(f"Termination condition: {results.solver.termination_condition}")

if results.solver.termination_condition in ['optimal', 'locallyOptimal']:
    if results.solver.termination_condition == 'locallyOptimal':
        print("WARNING: Solution found is only locally optimal.")
    print("Plotting results...")
    t_eval=[0.0, 60.0, 120.0, 180.0, 240.0, 300.0],
    x_eval=[0.0, 20.0, 30.0, 60.0, 80.0, 100.0, 110.0]
    fig = plot_results(model, t_eval=t_eval, x_eval=x_eval)
    plt.tight_layout()
    filename = "collector_pde_temp_profiles.png"
    plt.savefig(os.path.join(plot_dir, filename), dpi=150)
    plt.show()
    print_temp_profiles(model, t_eval=t_eval, x_eval=x_eval)
else:
    print("Solution not optimal. Check model formulation.")
    print(f"Solver message: {results.solver.message}")

## Inspect Solution

In [ ]:
T = pd.Series(model.T.extract_values())
T.index.names = ['t', 'x']
T = T.unstack()
T.shape

## Single Temperature Flow + Pipe Wall Temperature Model Simulation

In [ ]:
from solar_collector.solar_collector_dae_pyo_two_temp import (
    ZERO_C,
    create_pipe_flow_model,
    add_pde_constraints, 
    solve_model,
    plot_results,
    print_temp_profiles
)

In [ ]:
# Create and solve the model
print("Creating pipe flow heat transport model...")
model = create_pipe_flow_model(t_final=60.0*5)  # Simulate for 5 minutes

print("Adding PDE constraints...")
model = add_pde_constraints(model)

print("Solving the discretized PDE...")
results = solve_model(model, n_x=110, n_t=50, tol=1e-6)

print(f"Solver status: {results.solver.status}")
print(f"Termination condition: {results.solver.termination_condition}")

if results.solver.termination_condition in ['optimal', 'locallyOptimal']:
    if results.solver.termination_condition == 'locallyOptimal':
        print("WARNING: Solution found is only locally optimal.")
    print("Plotting results...")
    fig = plot_results(
        model, 
        t_vals_to_plot=[0.0, 60.0, 120.0, 180.0, 240.0, 300.0],
        x_vals_to_plot=[0.0, 20.0, 30.0, 60.0, 80.0, 100.0, 110.0]
    )
    plt.tight_layout()
    filename = "collector_pde_temp_profiles.png"
    plt.savefig(os.path.join(plot_dir, filename), dpi=150)
    plt.show()
    print_temp_profiles(model)
else:
    print("Solution not optimal. Check model formulation.")
    print(f"Solver message: {results.solver.message}")